In [1]:
# assignment 6
# MSDS 422
# Professor Anil Chaturvedi

# https://github.com/ageron/handson-ml/blob/master/10_introduction_to_artificial_neural_networks.ipynb
# https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb
# https://github.com/ageron/handson-ml/blob/master/12_distributed_tensorflow.ipynb

# more info
# https://www.datacamp.com/community/tutorials/cnn-tensorflow-python
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

# really wonderful resource for noobs
# https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/

<div style="text-align: right"><b>pkg imports</b></div>

In [2]:
# basix
import numpy as np
import pandas as pd
import seaborn as sns
import math
import random
import time

# prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

#modeling
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense

# other

%matplotlib inline   
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<div style="text-align: right"><b>configs, table setup</b></div>

In [3]:
"""
for sci-kit:
‘identity’, no-op activation, useful to implement linear bottleneck, returns f(x) = x
‘logistic’, the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).
‘tanh’, the hyperbolic tan function, returns f(x) = tanh(x).
‘relu’, the rectified linear unit function, returns f(x) = max(0, x)

for tf/keras, the same, respectively:
linear
sigmoid
tanh
relu

also (with tf):
selu
"""

#sk_act_fun = "identity"
sk_act_fun = "logistic"
#sk_act_fun = "tanh"
#sk_act_fun = "relu"

tf_act_fun = "sigmoid"

In [4]:
pd.options.display.float_format = "{:,.4f}".format
RANDOM_SEED = 8675309
responseVar = "label"

In [5]:
# setup for consolidated table, later

table = {"model 1":{"pkg":"sci-kit","act'n fn":"logistic","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 2":{"pkg":"sci-kit","act'n fn":"logistic","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 3":{"pkg":"sci-kit","act'n fn":"logistic","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 4":{"pkg":"sci-kit","act'n fn":"logistic","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 5":{"pkg":"tf/keras","act'n fn":"sigmoid","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 6":{"pkg":"tf/keras","act'n fn":"sigmoid","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 7":{"pkg":"tf/keras","act'n fn":"sigmoid","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None},
         "model 8":{"pkg":"tf/keras","act'n fn":"sigmoid","nbr_layers":None,"nodes_p_layer":None,"processing_time":None,"trainset_acc":None,"testset_acc":None}
        }

pd.DataFrame(table).transpose().head(2)

,pkg,act'n fn,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,sci-kit,logistic,None,None,None,None,None
model 2,sci-kit,logistic,None,None,None,None,None


<div style="text-align: right"><b>defining things for model</b></div>

In [6]:
# activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# lambda x: 1 / (1 + np.exp(-x))

In [7]:
training_iters = 10
learning_rate = 0.01
b_size = 128

# MNIST data input (img shape: 28*28)
n_input = 28

# MNIST total classes (0-9 digits)
n_classes = 10

<div style="text-align: right"><b>data setup for scikit</b></div>

In [8]:
sk_mnist_train = pd.read_csv('train.csv')
sk_mnist_test = pd.read_csv('test.csv')
#sk_mnist = pd.concat([mnist_test, mnist_train.drop(columns = "label")]) # combines the rows of 2 datasets

In [9]:
print("le shape of le train is le : ", sk_mnist_train.shape)
print("le shape of le test        : ", sk_mnist_test.shape)

le shape of le train is le :  (42000, 785)
le shape of le test        :  (28000, 784)


In [10]:
# 'test' data is our future data
sk_future_data = sk_mnist_test
sk_mnist = sk_mnist_train
del(sk_mnist_test)
del(sk_mnist_train)

In [11]:
try:
    sk_mnist_test
except NameError as e:
    print(e) # meow
    
try:
    sk_mnist_train
except NameError as e:
    print(e) # meow

name 'sk_mnist_test' is not defined
name 'sk_mnist_train' is not defined


In [12]:
#X = mnist_scaled.loc[:, mnist_scaled.columns != responseVar]
#y = mnist_scaled[responseVar]
sk_X = sk_mnist.loc[:, sk_mnist.columns != responseVar]
sk_y = sk_mnist[responseVar]

In [14]:
pd.DataFrame(sk_mnist.describe()["pixel774"]).loc[["count","mean","std","min","max"],]

,pixel774
count,"42,000.0000"
mean,0.2193
std,6.3129
min,0.0000
max,254.0000


In [16]:
scaler = StandardScaler()
sk_np_mnist_x = scaler.fit_transform(sk_X)
sk_X_scaled = pd.DataFrame(sk_np_mnist_x, index=sk_X.index, columns=sk_X.columns)

In [17]:
pd.DataFrame(sk_X_scaled.describe()["pixel774"]).loc[["count","mean","std","min","max"],]

,pixel774
count,"42,000.0000"
mean,0.0000
std,1.0000
min,-0.0347
max,40.2009


In [28]:
(254-0.2193)/6.3129

40.20033582030445

In [27]:
sk_X["pixel774"].describe()

count   42,000.0000
mean         0.2193
std          6.3129
min          0.0000
25%          0.0000
50%          0.0000
75%          0.0000
max        254.0000
Name: pixel774, dtype: float64

In [25]:
sk_X_scaled["pixel774"].unique()

array([-3.47365998e-02,  3.06964051e+01,  3.37061561e+01,  4.00424740e+01,
        1.54892422e+01,  9.62814818e+00,  3.98840661e+01,  3.02211813e+01,
        4.24227797e+00,  9.15711083e-01,  2.48353111e+01,  2.13503363e+01,
        4.40068592e+00,  1.70733217e+01,  7.72725281e+00,  3.13342234e+00,
        5.98476539e+00,  3.57654595e+01,  2.02414806e+01,  6.45998923e+00,
        9.46974023e+00,  2.81660645e+00,  1.50140184e+01,  1.29547151e+01,
        9.94496407e+00,  6.77680513e+00,  4.40487242e-01,  3.44981959e+01,
        1.65980978e+01,  1.18458594e+01,  5.66794950e+00,  3.03795892e+01,
        3.35477482e+01,  4.02008820e+01,  2.89539177e+01,  1.84989932e+01,
        1.56476502e+01,  1.40635707e+01,  3.22804846e+01,  1.97662568e+01,
        2.65819850e+00,  3.90920263e+01,  3.00627734e+01,  2.38848634e+01,
        6.93521307e+00,  1.75485455e+01,  1.53308343e+01,  3.70327230e+01,
        3.94088422e+01,  1.32715310e+01,  2.97501440e+00,  2.03998886e+01,
        1.26378992e+01,  

In [21]:
plt.hist(sk_X_scaled["pixel774"])
plt.show()

KeyboardInterrupt: 

Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x143a77320>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/matplotlib/transforms.py", line 178, in <lambda>
    ref = weakref.ref(self, lambda ref, sid=id(self),
KeyboardInterrupt


Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x116d378c0> (for post_execute):


KeyboardInterrupt: 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in callback <function flush_figures at 0x116d3ddd0> (for post_execute):
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/core/events.py", line 88, in trigger
    func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.7/site-packages/ipykernel/pylab/backend_inline.py", line 117, in flush_figures
    return show(True)
  File "/opt/anaconda3/lib/python3.7/site-packages/ipykernel/pylab/backend_inline.py", line 39, in show
    metadata=_fetch_figure_metadata(figure_manager.canvas.figure)
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/core/display.py", line 306, in display
    format_dict, md_dict = format(obj, include=include, exclude=exclude)
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython/core/formatters.py", line 180, in format
    data = formatter(obj)
  File "</opt/anaconda3/lib/python3.7/site-packages/decorator.py:decorator-gen-9>", line 2, in __call__
  File "/opt/anaconda3/lib/python3.7/site-packages/IPython

KeyboardInterrupt: 

In [16]:
sk_X_train, sk_X_test, sk_y_train, sk_y_test = train_test_split(sk_X_scaled, sk_y,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    shuffle=True,
                                                    random_state=RANDOM_SEED)

<div style="text-align: right"><b>data setup for tensorflow</b></div>

In [29]:
from keras.datasets import mnist
(tf_X_train, tf_y_train), (tf_X_test, tf_y_test) = mnist.load_data()

In [30]:
tf_X_valid, tf_X_train = tf_X_train[:5000] / 255.0, tf_X_train[5000:] / 255.0
tf_y_valid, tf_y_train = tf_y_train[:5000], tf_y_train[5000:]
tf_X_test = tf_X_test / 255.0

<div style="text-align: right"><b>model 1 : 2x1. 2 layers w 1 neuron each. sci-kit</b></div>

In [19]:
print("approximate start time : ", time.asctime())
start1 = time.perf_counter()

approximate start time :  Sat Oct 24 14:11:48 2020


In [20]:
twox1 = MLPClassifier(hidden_layer_sizes = (2, 1), # neurons comma layers
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [21]:
stop1 = time.perf_counter()
tm1 = round(stop1 - start1)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm1)
print("time of execution in minutes : ", round(tm1/60, 1))

approximate completion time :  Sat Oct 24 14:12:51 2020
time of execution in seconds :  63
time of execution in minutes :  1.1


In [22]:
table["model 1"]["nbr_layers"] = "1"
table["model 1"]["nodes_p_layer"] = "2"
table["model 1"]["processing_time"] = tm1
table["model 1"]["testset_acc"] = twox1.score(sk_X_test, sk_y_test)
table["model 1"]["trainset_acc"] = twox1.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 2 : 2x2. 2 layers w 2 neurons each. I think. sci-kit</b></div>

In [23]:
print("approximate start time : ", time.asctime())
start2 = time.perf_counter()

approximate start time :  Sat Oct 24 14:12:51 2020


In [24]:
twox2 = MLPClassifier(hidden_layer_sizes = (2, 2), 
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [25]:
stop2 = time.perf_counter()
tm2 = round(stop2 - start2)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm2)
print("time of execution in minutes : ", round(tm2/60, 1))

approximate completion time :  Sat Oct 24 14:13:55 2020
time of execution in seconds :  63
time of execution in minutes :  1.1


In [26]:
table["model 2"]["nbr_layers"] = "2"
table["model 2"]["nodes_p_layer"] = "2"
table["model 2"]["processing_time"] = tm2
table["model 2"]["testset_acc"] = twox2.score(sk_X_test, sk_y_test)
table["model 2"]["trainset_acc"] = twox2.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 3 : x1. 30 layers w 4 neurons each. sci-kit</b></div>

In [27]:
print("approximate start time : ", time.asctime())
start3 = time.perf_counter()

approximate start time :  Sat Oct 24 14:13:55 2020


In [28]:
thirtyx4 = MLPClassifier(hidden_layer_sizes = (30, 4), 
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

In [29]:
stop3 = time.perf_counter()
tm3 = round(stop3 - start3)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm3)
print("time of execution in minutes : ", round(tm3/60, 1))

approximate completion time :  Sat Oct 24 14:15:32 2020
time of execution in seconds :  97
time of execution in minutes :  1.6


In [30]:
table["model 3"]["nbr_layers"] = "30"
table["model 3"]["nodes_p_layer"] = "4"
table["model 3"]["processing_time"] = tm3
table["model 3"]["testset_acc"] = thirtyx4.score(sk_X_test, sk_y_test)
table["model 3"]["trainset_acc"] = thirtyx4.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 4 : x1. 4 layers, each w 30 neurons. sci-kit</b></div>

In [31]:
print("approximate start time : ", time.asctime())
start4 = time.perf_counter()

approximate start time :  Sat Oct 24 14:15:32 2020


In [32]:
fourx30 = MLPClassifier(hidden_layer_sizes = (4, 30), 
                      activation = sk_act_fun, 
                      alpha = learning_rate,
                      batch_size = b_size).fit(sk_X_train,
                                               sk_y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [33]:
stop4 = time.perf_counter()
tm4 = round(stop4 - start4)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm4)
print("time of execution in minutes : ", round(tm4/60, 1))

approximate completion time :  Sat Oct 24 14:16:37 2020
time of execution in seconds :  65
time of execution in minutes :  1.1


In [34]:
table["model 4"]["nbr_layers"] = "4"
table["model 4"]["nodes_p_layer"] = "30"
table["model 4"]["processing_time"] = tm4
table["model 4"]["testset_acc"] = fourx30.score(sk_X_test, sk_y_test)
table["model 4"]["trainset_acc"] = fourx30.score(sk_X_train, sk_y_train)

<div style="text-align: right"><b>model 5 : 2x1. 2 layers w 1 neurons each</b></div>

In [35]:
print("approximate start time : ", time.asctime())
start5 = time.perf_counter()

approximate start time :  Sat Oct 24 14:16:37 2020


In [36]:
#classification MLP with two hidden layers
model5 = Sequential()
model5.add(Flatten(input_shape=[28, 28]))
model5.add(Dense(1, activation=tf_act_fun))
model5.add(Dense(1, activation=tf_act_fun))
model5.add(Dense(10, activation="softmax"))

In [37]:
model5.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 785       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20        
Total params: 807
Trainable params: 807
Non-trainable params: 0
_________________________________________________________________


In [38]:
model5.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [39]:
history5 = model5.fit(tf_X_train, tf_y_train, epochs=5,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/5
1719/1719 [==============================] - 2s 1ms/step - loss: 2.3045 - accuracy: 0.1123 - val_loss: 2.3005 - val_accuracy: 0.1126
Epoch 2/5
1719/1719 [==============================] - 2s 970us/step - loss: 2.3008 - accuracy: 0.1123 - val_loss: 2.2999 - val_accuracy: 0.1126
Epoch 3/5
1719/1719 [==============================] - 2s 984us/step - loss: 2.2997 - accuracy: 0.1123 - val_loss: 2.2984 - val_accuracy: 0.1126
Epoch 4/5
1719/1719 [==============================] - 2s 978us/step - loss: 2.2975 - accuracy: 0.1123 - val_loss: 2.2953 - val_accuracy: 0.1126
Epoch 5/5
1719/1719 [==============================] - 2s 1ms/step - loss: 2.2928 - accuracy: 0.1124 - val_loss: 2.2888 - val_accuracy: 0.1126


In [40]:
stop5 = time.perf_counter()
tm5 = round(stop5 - start5)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm5)
print("time of execution in minutes : ", round(tm5/60, 1))

approximate completion time :  Sat Oct 24 14:16:46 2020
time of execution in seconds :  9
time of execution in minutes :  0.1


In [41]:
table["model 5"]["nbr_layers"] = "2"
table["model 5"]["nodes_p_layer"] = "1"
table["model 5"]["processing_time"] = tm5
table["model 5"]["testset_acc"] = model5.evaluate(tf_X_test, tf_y_test)[1]
table["model 5"]["trainset_acc"] = model5.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 1s 854us/step - loss: 2.2891 - accuracy: 0.1123


<div style="text-align: right"><b>model 6 : 2x2. 2 layers w 2 neurons each</b></div>

In [42]:
print("approximate start time : ", time.asctime())
start6 = time.perf_counter()

approximate start time :  Sat Oct 24 14:16:48 2020


In [43]:
#classification MLP with two hidden layers
model6 = Sequential()
model6.add(Flatten(input_shape=[28, 28]))
model6.add(Dense(2, activation=tf_act_fun))
model6.add(Dense(2, activation=tf_act_fun))
model6.add(Dense(10, activation="softmax"))

In [44]:
model6.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1570      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                30        
Total params: 1,606
Trainable params: 1,606
Non-trainable params: 0
_________________________________________________________________


In [45]:
model6.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history6 = model6.fit(tf_X_train, tf_y_train, epochs=5,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/5
1719/1719 [==============================] - 2s 1ms/step - loss: 2.3078 - accuracy: 0.1151 - val_loss: 2.2974 - val_accuracy: 0.1126
Epoch 2/5
1719/1719 [==============================] - 2s 1ms/step - loss: 2.2955 - accuracy: 0.1124 - val_loss: 2.2918 - val_accuracy: 0.1126
Epoch 3/5
1719/1719 [==============================] - 2s 1ms/step - loss: 2.2875 - accuracy: 0.1272 - val_loss: 2.2804 - val_accuracy: 0.1476
Epoch 4/5
1719/1719 [==============================] - 2s 1ms/step - loss: 2.2711 - accuracy: 0.1957 - val_loss: 2.2563 - val_accuracy: 0.2412
Epoch 5/5
1719/1719 [==============================] - 2s 1ms/step - loss: 2.2369 - accuracy: 0.2261 - val_loss: 2.2082 - val_accuracy: 0.2082


In [46]:
stop6 = time.perf_counter()
tm6 = round(stop6 - start6)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm6)
print("time of execution in minutes : ", round(tm6/60, 1))

approximate completion time :  Sat Oct 24 14:16:58 2020
time of execution in seconds :  10
time of execution in minutes :  0.2


In [47]:
table["model 6"]["nbr_layers"] = "2"
table["model 6"]["nodes_p_layer"] = "2"
table["model 6"]["processing_time"] = tm6
table["model 6"]["testset_acc"] = model6.evaluate(tf_X_test, tf_y_test)[1]
table["model 6"]["trainset_acc"] = model6.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 1s 837us/step - loss: 2.2107 - accuracy: 0.2063


<div style="text-align: right"><b>model 7 : 30x4. 30 layers w 4 neurons each</b></div>

In [48]:
print("approximate start time : ", time.asctime())
start7 = time.perf_counter()

approximate start time :  Sat Oct 24 14:17:00 2020


In [49]:
#classification MLP with two hidden layers
model7 = Sequential()
model7.add(Flatten(input_shape=[28, 28]))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(4, activation=tf_act_fun))
model7.add(Dense(10, activation="softmax"))

In [50]:
model7.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 3140      
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_11 (Dense)             (None, 4)                

In [51]:
model7.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history7 = model7.fit(tf_X_train, tf_y_train, epochs=30,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/30
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3129 - accuracy: 0.1098 - val_loss: 2.3011 - val_accuracy: 0.1126
Epoch 2/30
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3015 - accuracy: 0.1123 - val_loss: 2.3006 - val_accuracy: 0.1126
Epoch 3/30
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3015 - accuracy: 0.1123 - val_loss: 2.3011 - val_accuracy: 0.1126
Epoch 4/30
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3014 - accuracy: 0.1123 - val_loss: 2.3011 - val_accuracy: 0.1126
Epoch 5/30
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3015 - accuracy: 0.1123 - val_loss: 2.3011 - val_accuracy: 0.1126
Epoch 6/30
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3014 - accuracy: 0.1123 - val_loss: 2.3011 - val_accuracy: 0.1126
Epoch 7/30
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3014 - accuracy: 0.1123 - val_loss: 2.3010 - val_accuracy:

In [52]:
stop7 = time.perf_counter()
tm7 = round(stop7 - start7)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm7)
print("time of execution in minutes : ", round(tm7/60, 1))

approximate completion time :  Sat Oct 24 14:18:30 2020
time of execution in seconds :  91
time of execution in minutes :  1.5


In [53]:
table["model 7"]["nbr_layers"] = "30"
table["model 7"]["nodes_p_layer"] = "4"
table["model 7"]["processing_time"] = tm7
table["model 7"]["testset_acc"] = model7.evaluate(tf_X_test, tf_y_test)[1]
table["model 7"]["trainset_acc"] = model7.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 2s 1ms/step - loss: 2.3013 - accuracy: 0.1123


<div style="text-align: right"><b>model 8 : 4x30. 4 layers w 30 neurons each</b></div>

In [54]:
print("approximate start time : ", time.asctime())
start8 = time.perf_counter()

approximate start time :  Sat Oct 24 14:18:32 2020


In [55]:
#classification MLP with two hidden layers
model8 = Sequential()
model8.add(Flatten(input_shape=[28, 28]))
model8.add(Dense(30, activation=tf_act_fun))
model8.add(Dense(30, activation=tf_act_fun))
model8.add(Dense(30, activation=tf_act_fun))
model8.add(Dense(30, activation=tf_act_fun))
model8.add(Dense(10, activation="softmax"))

In [56]:
model8.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 30)                23550     
_________________________________________________________________
dense_37 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_38 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_39 (Dense)             (None, 30)                930       
_________________________________________________________________
dense_40 (Dense)             (None, 10)                310       
Total params: 26,650
Trainable params: 26,650
Non-trainable params: 0
__________________________________________________

In [57]:
model8.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])
history8 = model7.fit(tf_X_train, tf_y_train, epochs=12,
                     validation_data=(tf_X_valid, tf_y_valid))

Epoch 1/12
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3015 - accuracy: 0.1123 - val_loss: 2.3006 - val_accuracy: 0.1126
Epoch 2/12
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3015 - accuracy: 0.1123 - val_loss: 2.3007 - val_accuracy: 0.1126
Epoch 3/12
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3014 - accuracy: 0.1123 - val_loss: 2.3011 - val_accuracy: 0.1126
Epoch 4/12
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3014 - accuracy: 0.1123 - val_loss: 2.3010 - val_accuracy: 0.1126
Epoch 5/12
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3015 - accuracy: 0.1123 - val_loss: 2.3007 - val_accuracy: 0.1126
Epoch 6/12
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3014 - accuracy: 0.1123 - val_loss: 2.3007 - val_accuracy: 0.1126
Epoch 7/12
1719/1719 [==============================] - 3s 2ms/step - loss: 2.3014 - accuracy: 0.1123 - val_loss: 2.3009 - val_accuracy:

In [58]:
stop8 = time.perf_counter()
tm8 = round(stop8 - start8)
print("approximate completion time : ", time.asctime())
print("time of execution in seconds : ", tm8)
print("time of execution in minutes : ", round(tm8/60, 1))

approximate completion time :  Sat Oct 24 14:19:07 2020
time of execution in seconds :  35
time of execution in minutes :  0.6


In [59]:
table["model 8"]["nbr_layers"] = "4"
table["model 8"]["nodes_p_layer"] = "30"
table["model 8"]["processing_time"] = tm7
table["model 8"]["testset_acc"] = model8.evaluate(tf_X_test, tf_y_test)[1]
table["model 8"]["trainset_acc"] = model8.evaluate(tf_X_train, tf_y_train)[1]

1719/1719 [==============================] - 1s 872us/step - loss: 2.6079 - accuracy: 0.0907


In [60]:
print("processing time in seconds : ")
pd.DataFrame(table).transpose()

processing time in seconds : 


,pkg,act'n fn,nbr_layers,nodes_p_layer,processing_time,trainset_acc,testset_acc
model 1,sci-kit,logistic,1,2,63,0.3212,0.3190
model 2,sci-kit,logistic,2,2,63,0.5879,0.5687
model 3,sci-kit,logistic,30,4,97,0.9965,0.9017
model 4,sci-kit,logistic,4,30,65,0.8682,0.8262
model 5,tf/keras,sigmoid,2,1,9,0.1123,0.1135
model 6,tf/keras,sigmoid,2,2,10,0.2063,0.2043
model 7,tf/keras,sigmoid,30,4,91,0.1123,0.1135
model 8,tf/keras,sigmoid,4,30,91,0.0907,0.0892
